In [1]:
# plotting
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# numeric
import numpy as np
from numpy import random
import pandas as pd

# utils
from tqdm import tqdm_notebook as tqdm
from collections import Counter
import tempfile
import logging
import time
import datetime
import random

from collections import OrderedDict
from IPython.display import display
from pprint import pprint

# logging
logger = log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
logging.basicConfig()
log.info('%s logger started.', __name__)

INFO:__main__:__main__ logger started.


In [2]:
import argparse
import os
import sys
import gym
from gym import wrappers
import random
import numpy as np

import torch
import torch.optim as optim
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
import os
os.sys.path.append(os.path.abspath('.'))
%reload_ext autoreload
%autoreload 2

# Data

In [4]:
from src.environments.portfolio import PortfolioEnv, sharpe, max_drawdown

df_train = pd.read_hdf('./data/poloniex_30m.hf',key='train')
df_test = pd.read_hdf('./data/poloniex_30m.hf',key='test')

In [5]:
window_length = 50
seed = 0
asset_names = df_train.columns.levels[0].tolist()

In [6]:
# from sklearn.utils import shuffle
def unison_shuffled_copies(a, b, random_state=0):
    assert len(a) == len(b)
    np.random.seed(random_state)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def random_shift(x, fraction):
    """Apply a random shift to a pandas series."""
    min_x, max_x = np.min(x), np.max(x)
    m = np.random.uniform(-fraction, fraction, size=x.shape) + 1
    return np.clip(x * m, min_x, max_x)

def make_samples(df_train, augument=9):
    # make a window of X for each step
    X = []
    for i in tqdm(range(window_length+1, len(df_train))):
        data_window = df_train.iloc[i - window_length - 1:i].copy()

        # normalise by second to last open price (last will become y)
        open = data_window.xs('open', axis=1, level='Price')
        data_window = data_window.divide(open.iloc[-2], level='Pair')
        data_window = data_window.drop('open', axis=1, level='Price')
        obs = np.array([data_window[asset].as_matrix() for asset in asset_names])

        X.append(obs)

    X = np.array(X)

    # split last return off as label
    y = X[:,:,-1,0]
    X = X[:,:,:-1,:]
    
    if augument>0:
        X_old = X
        y_old = y
        X = []
        y = []
        for i in range(5):
            X.append(random_shift(X_old.copy(), augument))
            y.append(y_old.copy())
        
        X=np.concatenate(X)
        y=np.concatenate(y)
    
    X_train, y_train = unison_shuffled_copies(X, y, random_state=seed)
    
    print(X.shape,y.shape)
    return X, y

X_train, y_train = make_samples(df_train[:1000], augument=0.0025)

Widget Javascript not detected.  It may not be installed or enabled properly.



(4745, 6, 50, 3) (4745, 6)


# Model

In [7]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, BatchNormalization
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, BatchNormalization, Conv1D, InputLayer, Dropout, regularizers, Conv2D, Reshape
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.layers import Input, merge, Reshape
from keras.layers import concatenate, Conv2D
from keras.regularizers import l2, l1_l2
from keras.models import Model

Using TensorFlow backend.


In [8]:
nb_actions=y_train.shape[1]
reg=1e-8
observation_space = X_train.shape[1:]

# Simple CNN model model
model = Sequential()
model.add(InputLayer(input_shape=observation_space))
# model.add(Reshape(observation_space))
model.add(Conv2D(
    filters=2,
    kernel_size=(1,3),
    kernel_regularizer=l2(reg),
    activation='elu'
))
model.add(BatchNormalization()) # lets add batch norm to decrease training time

model.add(Conv2D(
    filters=20,
    kernel_size=(1,window_length-2),
    kernel_regularizer=l2(reg),
    activation='elu'
))
model.add(BatchNormalization())

model.add(Conv2D(
    filters=1,
    kernel_size=(1,1),
    kernel_regularizer=l2(reg),
    activation='elu'
))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(
    nb_actions, 
    kernel_regularizer=l2(reg)
)) # this adds cash bias
model.add(Activation('softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6, 50, 3)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 48, 2)          20        
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 48, 2)          8         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 1, 20)          1940      
_________________________________________________________________
batch_normalization_2 (Batch (None, 6, 1, 20)          80        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 1, 1)           21        
_________________________________________________________________
batch_normalization_3 (Batch (None, 6, 1, 1)           4         
__________

# Train

In [ ]:
model.compile('adam',loss='mse', metrics=['mae'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=25,
          epochs=50,
          verbose=1,
          validation_split=0.15,
          callbacks=[]
         )

# Test